<font size=6>**面向对象高级编程**<font/>

数据封装、继承和多态只是面向对象程序设计中最基础的3个概念。在Python中，面向对象还有很多高级特性，允许我们写出非常强大的功能。

我们会讨论多重继承、定制类、元类等概念。

# 使用\_\_slots\_\_
正常情况下，当我们定义了一个class，创建了一个class的实例后，我们可以给该实例绑定任何属性和方法，这就是动态语言的灵活性。先定义class：

In [1]:
class Student(object):
    pass

然后尝试给实例绑定属性

In [2]:
s=Student()
s.name='Micheal'
print(s.name)

Micheal


还可以尝试给实例绑定一个方法。

In [3]:
def set_age(self,age):
    self.age=age

In [5]:
from types import MethodType
s.set_age=MethodType(set_age,s)#给实例绑定方法
s.set_age(10)
print(s.age)

10


但是，给一个实例绑定的方法，对另一个实例是不起作用的

In [6]:
s2=Student()
s2.set_age(25)

AttributeError: 'Student' object has no attribute 'set_age'

为了给所有实例都绑定方法，可以给class绑定方法：

In [7]:
def set_score(self,score):
    self.score=score

In [19]:
Student.set_score=set_score #动态绑定类方法可以直接赋值

给类绑定方法后，所有实例都可调用。

In [9]:
s.set_score(70)
s2.set_score(87)
s2.score

87

In [10]:
def set_gender(self,gender):
    self.gender=gender

In [11]:
s.set_gender=set_gender
s.set_gender('male')
s.gender

TypeError: set_gender() missing 1 required positional argument: 'gender'

In [12]:
s.set_gender(s,'male')
s.gender

'male'

使用赋值语句和MethodType均可以绑定方法。
但是使用MethodType绑定方法，调用方法时会将被绑定对象作为参数自动传入方法。
使用赋值语句绑定方法，不会自动传参。但是值得注意的是，如果是类绑定方法，那么他自身使用该方法时不会自动传参，但是他的对象也拥有该方法，而且使用时会自动传参。

___

但是，如果我们想要限制实例的属性怎么办？比如，只允许对Student实例添加name和age属性。

为了达到限制的目的，Python允许在定义class的时候，定义一个特殊的\_\_slots\_\_变量，来限制该class实例能添加的属性：

In [13]:
class Student(object):
    __slots__=('name','age') # 用tuple定义允许绑定的属性名称

In [14]:
s=Student()
s.name='Michael'
s.age=25
s.score=78

AttributeError: 'Student' object has no attribute 'score'

由于'score'没有被放到\_\_slots\_\_中，所以不能绑定score属性，试图绑定score将得到AttributeError的错误。

使用\_\_slots\_\_要注意，\_\_slots\_\_定义的属性**仅对当前类实例起作用**，对继承的子类是不起作用的：

In [15]:
class GraduateStudent(Student):
    pass

In [16]:
g=GraduateStudent()
g.score=100
g.score

100

除非在子类中也定义\_\_slots\_\_，这样，子类实例允许定义的属性就是自身的\_\_slots\_\_加上父类的\_\_slots\_\_

In [17]:
class SmallStudent(Student):
    __slots__=('score','phonenumber')

In [18]:
small_student=SmallStudent()
small_student.name='Michael'
small_student.age=11
small_student.score=89
small_student.phonenumber='172681'
small_student.gender='male'

AttributeError: 'SmallStudent' object has no attribute 'gender'

## 小练习
* s.set_age=MethodType(set_age,s)和setattr(s,'set_age',set_age)有什么区别？<br/>
setattr是先检查Student类有无 ‘set_age’方法，没有的话再加上去。  